In [1]:
import sys
import itertools

import numpy as np
import pandas as pd
import scipy
from scipy.signal import savgol_filter

import neuroseries as nts

from progressbar import ProgressBar

import time
import matplotlib.pyplot as plt


import seaborn as sns 
import bk.load
import bk.signal
import bk.plot
import importlib; 
importlib.reload(bk.signal)
importlib.reload(bk.plot)
%matplotlib qt
import basefunction.anass_load as at

In [4]:
path = "Z:\Rat08\Rat08-20130713"

bk.load.current_session(path)

laps = scipy.io.loadmat(bk.load.session+'-LapType.mat')
run = scipy.io.loadmat(bk.load.session + '-TrackRunTimes.mat')

danger_laps = laps['aplaps'][0]['run'][0]
danger_laps = nts.IntervalSet(danger_laps[:,0],danger_laps[:,1],time_units = 's')

safe_laps = laps['safelaps'][0]['run'][0]
safe_laps = nts.IntervalSet(safe_laps[:,0],safe_laps[:,1],time_units = 's')

trackruntimes = run['trackruntimes']
trackruntimes = nts.IntervalSet(trackruntimes[:,0],trackruntimes[:,1],time_units = 's')

laps = {'run':trackruntimes,'danger':danger_laps,'safe':safe_laps}
states = bk.load.states(bk.load.session)

lfp = bk.load.loadLFP(bk.load.path+ '\\' + bk.load.session+".lfp",166,13)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713


In [6]:
del states['drowsy']

lfps = {}
for lap in laps:
    lfp_ = lfp.restrict(laps[lap])
    lfps.update({lap:lfp_})

for state in states:
    lfp_ = lfp.restrict(states[state])
    lfps.update({state:lfp_})

In [12]:
filtered = bk.signal.passband(lfp,5,10)
xa = scipy.signal.hilbert(filtered)
xa = nts.Tsd(np.array(filtered.index),xa)
x_power = scipy.stats.zscore(np.abs(xa.values)**2)
x_power = nts.Tsd(np.array(filtered.index),x_power)

x_phase = np.angle(xa.values)
x_phase = nts.Tsd(np.array(filtered.index),x_phase)

x_rem = x_power.restrict(states['Rem'])
x_sws = x_power.restrict(states['sws'])

f_rem = scipy.signal.savgol_filter(x_rem.values,3001,2)
f_sws = scipy.signal.savgol_filter(x_sws.values,3001,2)


plt.plot(x_rem.as_units('s').index,f_rem)
plt.plot(x_sws.as_units('s').index,f_sws)

In [280]:
p = ['run','safe','danger']

for l in p:
    print(l)
    f, Pxx_den = scipy.signal.welch(lfps[l], 1250,nperseg=1024,nfft=1250*10,scaling='density')
    plt.plot(f, Pxx_den)
    plt.xlim(0,100)
    plt.xlabel('frequency [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.show()
plt.xlim(2,20)
plt.legend([i for i in p])

run
safe
danger


In [281]:
lfps.keys()

dict_keys(['run', 'danger', 'safe', 'Rem', 'sws', 'wake'])

In [282]:
def thetaMap(lfp,pos):
    filt_lfp = bk.signal

In [283]:
path = bk.load.sessions()['Path'][5]
bk.load.current_session(path)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713


True

In [30]:
lfp = bk.load.loadLFP(bk.load.path+ '\\' + bk.load.session+".lfp",166,101)

In [287]:
states = bk.load.states(bk.load.session)

In [288]:
sub = plt.subplot(3,1,1)
plt.plot(lfp.as_units('s')[3200:3400])
plt.plot(filtered.as_units('s')[3200:3400])
bk.plot.intervals(states['Rem'],'orange')
bk.plot.intervals(states['sws'],'grey')
plt.subplot(3,1,2,sharex = sub)
plt.plot(x_power.as_units('s')[3200:3400])
plt.subplot(3,1,3,sharex = sub)
plt.plot(x_phase.as_units('s')[3200:3400])
# bk.plot.intervals(states['Rem'],'orange',time_units='us')
# bk.plot.intervals(states['sws'],'grey',time_units='us')

In [289]:
f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=2500,noverlap=1250)
mask = (3000<t) & (t<3500)
plt.pcolormesh(t[mask], f, Sxx[:,mask])
plt.clim(0,20000)
plt.ylim(0,80)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

Rem
sws
wake
drowsy


In [32]:
lfp_rem = lfp.restrict(states['Rem'])

In [355]:
# f_rem = np.convolve(x_rem.values,k,'same')
# f_sws = np.convolve(x_sws.values,k,'same')


In [33]:
sub = plt.subplot(3,1,1)
plt.plot(lfp_rem)
f = bk.signal.passband(lfp_rem,5,10)
plt.plot(bk.signal.passband(lfp_rem,5,10))
plt.subplot(3,1,2 ,sharex = sub)
xa = scipy.signal.hilbert(f)
xa = nts.Tsd(np.array(f.index),xa)
x_phase = np.angle(xa.values)
x_phase = nts.Tsd(np.array(f.index),x_phase)

x_power = 
plt.plot(x_phase)